# Question Answering in Documenti

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic import hub
from langchain_chroma import Chroma

d:\AppPython\pinokio\bin\miniconda\envs\datamaster\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = TextLoader('data/dialoghi/dialogo.txt', encoding="utf-8")
documents = loader.load()

In [3]:
documents

[Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Al negozio di abbigliamento\n\nCommesso – Buongiorno, posso aiutarla?\nCliente – Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?\nCommesso – Certo, che taglia?\nCliente – Porto una 50.\nCommesso – Eccoli qua!\n\nCliente – Grazie… però prima, posso dire che finalmente oggi si respira un po’? Ieri sembrava luglio!\nCommesso – Ah, guardi, non me ne parli. Stamattina ho già cambiato due volte la maglietta, da quanto fa caldo. E poi ieri sera quell’umidità… sembrava di stare in una serra.\nCliente – Già, e poi è arrivato quel temporale all’improvviso, proprio mentre tornavo a casa. Mi sono bagnato fino alle ossa.\nCommesso – Eh sì, classico. Qui appena arriva un po’ di caldo, dopo due giorni parte il temporale estivo. Però oggi sembra tenere, almeno per ora.\nCliente – Speriamo, che devo ancora fare un giro in centro dopo.\n\nCommesso – A proposito, è appassionato di sport? Ieri notte ho gua

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("openai_key"))

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=75)

'''
Alcuni altri splitter standard:
* CharacterTextSplitter - split fissi su un set di caratteri
* NLTKTextSplitter - usa la libreria NLTK per identificare singole frasi da usare come chunk
* SpacyTextSplitter - usa la libreria Spacy per identificare singole frasi da usare come chunk
* MarkdownTextSplitter - sfrutta la sintassi Markdown per identificare sezioni atomiche di testo da usare come chunk (intestazioni, liste, blocchi di codice, ...)
* LatexTextSplitter - sfrutta la sintassi Latex per identificare sezioni atomiche di testo da usare come chunk (sezioni, sottosezioni, equazioni)
'''
splits = text_splitter.split_documents(documents)

splits

[Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Al negozio di abbigliamento\n\nCommesso – Buongiorno, posso aiutarla?\nCliente – Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?\nCommesso – Certo, che taglia?\nCliente – Porto una 50.\nCommesso – Eccoli qua!'),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Cliente – Grazie… però prima, posso dire che finalmente oggi si respira un po’? Ieri sembrava luglio!'),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Commesso – Ah, guardi, non me ne parli. Stamattina ho già cambiato due volte la maglietta, da quanto fa caldo. E poi ieri sera quell’umidità… sembrava di stare in una serra.'),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Cliente – Già, e poi è arrivato quel temporale all’improvviso, proprio mentre tornavo a casa. Mi sono bagnato fino alle ossa.'),
 Document(metadata={'source': 'data/dialoghi/dialogo.t

In [6]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [7]:
from langchain_classic import hub

In [8]:
# Recupero di un prompt standard dall'Hub di LangChain/LangGraph
# (https://smith.langchain.com/hub)

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

d:\AppPython\pinokio\bin\miniconda\envs\datamaster\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [10]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
rag_chain.invoke("qual'è la taglia del cliente?")

'La taglia del cliente è 50.'

In [12]:
rag_chain.invoke("in che negozio è ambientata la scena?")

'La scena è ambientata in un negozio di abbigliamento.'

In [13]:
rag_chain.invoke("dove sono i camerini?")

'I camerini sono in fondo a destra, accanto alle scale.'

In [12]:
(retriever | format_docs).invoke("qual'è la taglia del cliente?")

'Commesso – Come vanno?\nCliente – Il modello mi piace, ma sono un po’ stretti. Posso provare una taglia più larga?\nCommesso – Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore. Vuole provare lo stesso modello in marrone?\n\nCliente – No, grazie, il marrone proprio non mi piace. Non avete altri colori in questa taglia?\nCommesso – Allora, nella taglia 52 abbiamo il marrone, il rosso e il grigio.\nCliente – Vabbè, li provo in grigio, vediamo come mi stanno.\n\nAl negozio di abbigliamento\n\nCommesso – Buongiorno, posso aiutarla?\nCliente – Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?\nCommesso – Certo, che taglia?\nCliente – Porto una 50.\nCommesso – Eccoli qua!\n\nCliente – Ho provato i pantaloni, anche in grigio sono proprio belli e la taglia è perfetta! Quanto costano?\nCommesso – Costano 85€.\nCliente – Ma non sono in sconto?'